# Chains in LangChain

Chains are easily reusable components linked together.

Chains encode a sequence of calls to components like models, document retrievers, other Chains, etc., and provide a simple interface to this sequence.

The Chain interface makes it easy to create apps that are:

* Stateful: add Memory to any Chain to give it state,

* Observable: pass Callbacks to a Chain to execute additional functionality, like logging, outside the main sequence of component calls,

* Composable: combine Chains with other components, including other Chains.

## Outline

* LLMChain
* Sequential Chains
  * SimpleSequentialChain
  * SequentialChain
* Router Chain

In [67]:
## Load you env variables
import os
from dotenv import load_dotenv,find_dotenv
import openai
import anthropic
from anthropic import Anthropic

_ = load_dotenv(find_dotenv(filename='.env'))

In [68]:
## Import Pandas Library
import pandas as pd

## read the cv file from Data folder
projectData = pd.read_csv('./Your_CSV_Data_File')
projectData.head()

FileNotFoundError: [Errno 2] No such file or directory: './Your_CSV_Data_File'

In [ ]:
## Import necessary Langchain's components to interact with the projectData
from langchain_community.llms import HuggingFaceEndpoint
from langchain.prompts import  ChatPromptTemplate
from langchain.chains import LLMChain
from langchain_openai import ChatOpenAI

HUGGINGFACEHUB_API_TOKEN = os.getenv("HUGGINGFACEHUB_API_TOKEN")

openai_llm_model = ChatOpenAI(
    model="gpt-3.5-turbo",
    api_key=os.getenv("OPENAI_API_KEY"),
    temperature=0.2,
    max_tokens=1024
)

microsoft_phi3_model = HuggingFaceEndpoint(
    repo_id = "microsoft/Phi-3-mini-4k-instruct",
    temperature=0.5,
    token=HUGGINGFACEHUB_API_TOKEN,
    add_to_git_credential=True
)


WARNING! token is not default parameter.
                    token was transferred to model_kwargs.
                    Please make sure that token is what you intended.
WARNING! add_to_git_credential is not default parameter.
                    add_to_git_credential was transferred to model_kwargs.
                    Please make sure that add_to_git_credential is what you intended.


The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to C:\Users\mehdi\.cache\huggingface\token
Login successful


## LLMChain

In [ ]:
## Construct the LLM Chain
question = "Who won the FIFA World Cup in the year 1994? "

## Chain Of thoughts Prompting
template = """Question: {question}

Answer: Let's think step by step."""

prompt = ChatPromptTemplate.from_template(template)

llm_chain = LLMChain(prompt=prompt,llm=microsoft_phi3_model)

c:\Users\mehdi\anaconda3\envs\langchain-tuto\lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 0.3.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  warn_deprecated(


In [ ]:
llm_chain.run(question)

c:\Users\mehdi\anaconda3\envs\langchain-tuto\lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


' The FIFA World Cup is an international soccer tournament that takes place every four years. The tournament in 1994 was held in the United States. The winner of the 1994 FIFA World Cup was Brazil. They defeated Italy in the final match with a score of 3-2.'

## SimpleSequentialChain - SequentialChain

Another Types of Chain that are aimed to combine multiple chains where the output of the one chain is the input of the next chain.

* SimpleSequentialChain:Single Input/Output.
* SequentialChain:multiple Inputs/Outpus.

### SimpleSequentialChain

In [ ]:
from langchain.chains import  SimpleSequentialChain
from langchain.output_parsers import JsonOutputToolsParser

product_category = "Building Muscles"

prompt_one = ChatPromptTemplate.from_template(
    """
    Try to generate me good specific keyword for a product that belongs to a category delimited by triple backticks.\
    Try to think step by step before ansering the final answer.\
    product_category : ```{product_category}```
"""
)
# prompt_one.messages[0].input_variables
first_chain = LLMChain(llm=openai_llm_model,prompt=prompt_one)
first_chain.invoke({
    "product_category":product_category
})

{'product_category': 'Building Muscles',
 'text': '1. Think about the product category: Building Muscles\n2. Consider what types of products are commonly associated with building muscles (e.g. protein supplements, weightlifting equipment, workout programs)\n3. Brainstorm keywords that are specific to products in the building muscles category\n4. Consider the target audience for these products and what keywords they might use when searching for them\n5. Think about the benefits or features of the product that would appeal to someone looking to build muscles\n6. Combine these elements to generate a specific keyword for a product in the building muscles category\n\nFinal keyword: "Muscle-building protein powder"'}

In [ ]:
prompt_two = ChatPromptTemplate.from_template(
    """
    Try to generate me a call to action text for a product based on their keywords delimited by triple backticks.\
    Try to think step by step before answering the final answer.\
    keywords : ```{product_keyword}```
"""
)
# prompt_one.messages[0].input_variables
second_chain = LLMChain(llm=openai_llm_model,prompt=prompt_two)

### Combining the chains to construct one SequentialChain

In [ ]:
sequetialChains = SimpleSequentialChain(chains=[first_chain,second_chain],verbose=True)
sequetialChains.run(product_category)



> Entering new SimpleSequentialChain chain...
1. Think about the main purpose or benefit of the product within the "Building Muscles" category.
2. Consider any unique features or ingredients that set the product apart.
3. Brainstorm related terms or synonyms that potential customers might use when searching for this type of product.
4. Combine these elements into a specific keyword that accurately describes the product.

Keyword: "Protein-packed Muscle Builder Supplement"
Are you looking to build muscles and increase your strength? Look no further than our Protein-packed Muscle Builder Supplement! With unique ingredients and powerful features, this supplement is designed to help you achieve your fitness goals. Don't wait any longer, try our Protein-packed Muscle Builder Supplement today and take your workouts to the next level!

> Finished chain.


"Are you looking to build muscles and increase your strength? Look no further than our Protein-packed Muscle Builder Supplement! With unique ingredients and powerful features, this supplement is designed to help you achieve your fitness goals. Don't wait any longer, try our Protein-packed Muscle Builder Supplement today and take your workouts to the next level!"

### SequentialChain

In [ ]:
from langchain.chains import SequentialChain

product_description = "Are you looking to build muscles and increase your strength? Look no further than our Protein-packed Muscle Builder Supplement! \
    With unique ingredients and powerful features, this supplement is designed to help you achieve your fitness goals.\
        Don't wait any longer, try our Protein-packed Muscle Builder Supplement today and take your workouts to the next level!"

first_prompt = ChatPromptTemplate.from_template(
    """
    You are an expert assistant in text translation .
    Try to translate into frensh the text delimited by triple backticks.
    text : ```{product_description}```
    """
)
chain_one = LLMChain(llm=microsoft_phi3_model,
                     prompt=first_prompt,
                     output_key="product_description_frensh_version")

In [ ]:
second_prompt = ChatPromptTemplate.from_template(
    """
    You are an expert assistant in text summurization .
    Try to summurize into specific manner the text delimited by triple backticks.
    text : ```{product_description_frensh_version}```
    """
)
chain_two = LLMChain(llm=openai_llm_model,
                     prompt=second_prompt,
                     output_key="summurized_product_description_frensh_version")

In [ ]:
third_prompt = ChatPromptTemplate.from_template(
    """
    You are an expert assistant in natural language model .
    Try to answer the question delimited by triple backticks.
    question : ```What is the language of this review:{product_description}```
    """
)
chain_three = LLMChain(llm=openai_llm_model,
                     prompt=third_prompt,
                     output_key="review_language")

In [ ]:
fourth_prompt = ChatPromptTemplate.from_template(
    """
    Try to write me a follow up feedback to the summary that is delimited by triple backticks \
          into the {review_language} language.
    summary : ```{summurized_product_description_frensh_version}```
    """
)
chain_four = LLMChain(llm=openai_llm_model,
                     prompt=fourth_prompt,
                     output_key="follow_up_feedback")

In [ ]:
combined_sequetial_chain = SequentialChain(
    chains=[chain_one,chain_two,chain_three,chain_four],
    input_variables=["product_description"],
    output_variables=[
        "product_description_frensh_version",
        "summurized_product_description_frensh_version",
        "follow_up_feedback"
        ],
    verbose=True
)
##combined_sequetial_chain.to_json()
combined_sequetial_chain.invoke(
    {
        "product_description":product_description
    }
)



> Entering new SequentialChain chain...

> Finished chain.


{'product_description': "Are you looking to build muscles and increase your strength? Look no further than our Protein-packed Muscle Builder Supplement!     With unique ingredients and powerful features, this supplement is designed to help you achieve your fitness goals.        Don't wait any longer, try our Protein-packed Muscle Builder Supplement today and take your workouts to the next level!",
 'product_description_frensh_version': "\n    Here is the translation :\n    Vous cherchez peut-être à développer vos muscles et augmenter votre force ? N'attendez plus, essayez notre Supplément Muscle Builder Proteiné aujourd'hui et améliorez votre niveau d'entrainement !\n\n",
 'summurized_product_description_frensh_version': 'Summary: The text promotes a muscle-building protein supplement to help increase muscle mass and strength during workouts.',
 'follow_up_feedback': 'Thank you for providing a summary of the text. It seems like the focus is on promoting a muscle-building protein supple

## Router Chain 

* Dynamically route logic based on input

* Routing allows you to create non-deterministic (strict feed) chains where the output of a previous step defines the next step. Routing helps provide structure and consistency around interactions with LLMs.

In [70]:
## Prepare differents prompt template for each use case
physics_template = """You are a very smart physics professor. \
You are great at answering questions about physics in a concise\
and easy to understand manner. \
When you don't know the answer to a question you admit\
that you don't know.

Here is a question:
{input}"""


math_template = """You are a very good mathematician. \
You are great at answering math questions. \
You are so good because you are able to break down \
hard problems into their component parts, 
answer the component parts, and then put them together\
to answer the broader question.

Here is a question:
{input}"""

history_template = """You are a very good historian. \
You have an excellent knowledge of and understanding of people,\
events and contexts from a range of historical periods. \
You have the ability to think, reflect, debate, discuss and \
evaluate the past. You have a respect for historical evidence\
and the ability to make use of it to support your explanations \
and judgements.

Here is a question:
{input}"""


computerscience_template = """ You are a successful computer scientist.\
You have a passion for creativity, collaboration,\
forward-thinking, confidence, strong problem-solving capabilities,\
understanding of theories and algorithms, and excellent communication \
skills. You are great at answering coding questions. \
You are so good because you know how to solve a problem by \
describing the solution in imperative steps \
that a machine can easily interpret and you know how to \
choose a solution that has a good balance between \
time complexity and space complexity. 

Here is a question:
{input}"""

In [71]:
## Provide configuration for each prompt template
prompt_infos = [
    {
        "name": "physics", 
        "description": "Good for answering questions about physics", 
        "prompt_template": physics_template
    },
    {
        "name": "math", 
        "description": "Good for answering math questions", 
        "prompt_template": math_template
    },
    {
        "name": "History", 
        "description": "Good for answering history questions", 
        "prompt_template": history_template
    },
    {
        "name": "computer science", 
        "description": "Good for answering computer science questions", 
        "prompt_template": computerscience_template
    }
]

In [96]:
from langchain.chains.router import MultiPromptChain
from langchain.chains.router.llm_router import LLMRouterChain, RouterOutputParser
from langchain.prompts import PromptTemplate
import json

destination_chain = {}
for p_infos in prompt_infos:
    name = p_infos["name"]
    prompt_template = p_infos["prompt_template"]
    prompt = ChatPromptTemplate.from_template(template=prompt_template)
    chain = LLMChain(llm=openai_llm_model,prompt=prompt)
    ## Append chains configuration to destination_chain dict
    destination_chain[name] = chain

destination = [f"{p['name']}:{p['description']}" for p in prompt_infos]
destination_str = "\n".join(destination)
destination_str

'physics:Good for answering questions about physics\nmath:Good for answering math questions\nHistory:Good for answering history questions\ncomputer science:Good for answering computer science questions'

In [88]:
## Create a default chain for the LLM can't choose between the crafted chains
default_prompt = ChatPromptTemplate.from_template("{input}")
default_chain = LLMChain(llm=openai_llm_model,prompt=default_prompt)

In [97]:
## The template that will be injected into the LLMrouterChain
MULTI_PROMPT_ROUTER_TEMPLATE = """Given a raw text input to a \
language model select the model prompt best suited for the input. \
You will be given the names of the available prompts and a \
description of what the prompt is best suited for. \
You may also revise the original input if you think that revising\
it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}}}
```

REMEMBER: "destination" MUST be one of the candidate prompt \
names specified below OR it can be "DEFAULT" if the input is not\
well suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input \
if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
{destinations}

<< INPUT >>
{{input}}

<< OUTPUT (remember to include the ```json``` formatting)>>"""

In [99]:
router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(
    destinations=destination_str
)
router_prompt = PromptTemplate(
    template=router_template,
    input_variables=["input"],
    output_parser=RouterOutputParser()
)
router_chain = LLMRouterChain.from_llm(
    llm=openai_llm_model,
    prompt=router_prompt
)

LLMRouterChain(llm_chain=LLMChain(prompt=PromptTemplate(input_variables=['input'], output_parser=RouterOutputParser(), template='Given a raw text input to a language model select the model prompt best suited for the input. You will be given the names of the available prompts and a description of what the prompt is best suited for. You may also revise the original input if you think that revisingit will ultimately lead to a better response from the language model.\n\n<< FORMATTING >>\nReturn a markdown code snippet with a JSON object formatted to look like:\n```json\n{{\n    "destination": string \\ name of the prompt to use or "DEFAULT"\n    "next_inputs": string \\ a potentially modified version of the original input\n}}\n```\n\nREMEMBER: "destination" MUST be one of the candidate prompt names specified below OR it can be "DEFAULT" if the input is notwell suited for any of the candidate prompts.\nREMEMBER: "next_inputs" can just be the original input if you don\'t think any modificati

In [105]:
multi_router_chain = MultiPromptChain(
      router_chain=router_chain,
      destination_chains=destination_chain,
      default_chain=default_chain,
      verbose=True
)
multi_router_chain.invoke({
    "input":"What is thermodynamic ?"
})



> Entering new MultiPromptChain chain...
physics: {'input': 'What is thermodynamics?'}
> Finished chain.


{'input': 'What is thermodynamics?',
 'text': 'Thermodynamics is the branch of physics that deals with the relationships between heat and other forms of energy. It studies how energy is transferred as heat and work, and how these processes affect the properties of matter. Thermodynamics also explores concepts such as temperature, entropy, and the laws of thermodynamics, which govern the behavior of energy in systems.'}